In [26]:
import numpy as np
import psycopg2
from psycopg2 import OperationalError
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)
psycopg2.extensions.register_adapter(np.bool_, psycopg2._psycopg.AsIs)
from pytrends.request import TrendReq
from time import sleep
import pandas as pd
import seaborn as sns
import json

In [27]:
#get credentials
with open("postgre_credentials.json") as file:
    data = json.load(file)

In [28]:
def reset_connection():
    if 'connection' not in locals():
        connection = psycopg2.connect(
        database = data["db_name"], 
        user = data["db_user"],
        password = data["db_password"],
        host = data["db_host"],
        port = data["db_port"]
        )
        cur = connection.cursor()
        return connection,cur;
    else:
        connection.close()
        connection = psycopg2.connect(
        database = data["db_name"], 
        user = data["db_user"],
        password = data["db_password"],
        host = data["db_host"],
        port = data["db_port"]
        )
        cur = connection.cursor()
        return connection, cur;

In [29]:
connection, cur = reset_connection()

## Get 10 shows that have finished airing

In [35]:
query="""SELECT anime_id
FROM anime
WHERE air_status LIKE 'Finished Airing'
"""
cur.execute(query)
anime_ids = cur.fetchall()

In [40]:
test_show = anime_ids[0][0]

In [41]:
def call_google_trends(country, anime_id):    
    try:                
        pytrends.build_payload(kw_list=[name],cat=0,timeframe='all', geo=country)
        data = pytrends.interest_over_time()
        #print(data)
        if not data.empty:
            data = data.drop(labels=['isPartial'],axis='columns')
            return data
    except (RuntimeError):
        print("Invalid Name?")
        print(RuntimeError)

In [42]:
connection.close()